In [ ]:
# mexc_api.py
import time
import hmac
import hashlib
import requests

class MexcAPI:
    def __init__(self, api_key, api_secret, base_url='https://contract.mexc.com'):
        """
        Initialize the MEXC API client.
        
        :param api_key: Your API key.
        :param api_secret: Your API secret.
        :param base_url: The base URL for the API endpoints.
        """
        self.api_key = api_key
        self.api_secret = api_secret
        self.base_url = base_url

    def create_signature(self, params):
        """
        Generate an HMAC-SHA256 signature using the provided parameters.
        
        :param params: A dictionary of the parameters.
        :return: The hexadecimal signature string.
        """
        # Sort parameters by key and join them into a query string.
        query_string = '&'.join(f"{key}={value}" for key, value in sorted(params.items()))
        signature = hmac.new(
            self.api_secret.encode('utf-8'),
            query_string.encode('utf-8'),
            hashlib.sha256
        ).hexdigest()
        return signature

    def send_order(self, symbol, price, quantity, side, order_type="LIMIT"):
        """
        Send an order to MEXC.
        
        :param symbol: The trading pair (e.g., "BTC_USDT").
        :param price: The price at which to execute the order.
        :param quantity: The order volume.
        :param side: Order side ("1" for buy, "2" for sell; check the docs).
        :param order_type: The type of order, such as "LIMIT" or "MARKET".
        :return: The JSON response from the API.
        """
        endpoint = '/api/v1/private/order'
        url = self.base_url + endpoint

        # Prepare order parameters.
        params = {
            "symbol": symbol,
            "price": price,
            "vol": quantity,  # Some APIs use "vol" to denote volume.
            "side": side,
            "type": order_type,
            "timestamp": str(int(time.time() * 1000)),
            "api_key": self.api_key
        }

        # Generate and add the signature.
        params["sign"] = self.create_signature(params)

        # Send the POST request.
        response = requests.post(url, data=params)

        try:
            return response.json()
        except Exception as e:
            return {"error": "Failed to parse response", "details": str(e)}
